In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, log_loss
from xgboost import XGBClassifier
import pickle

In [2]:
call = pd.read_csv("../data/Train_call.txt", sep='\t')
clin = pd.read_csv("../data/Train_clinical.txt", sep='\t')

In [3]:
X=call[clin[clin['Subgroup']!='HER2+']['Sample']]
X=X.T.reset_index(drop=True)

y = clin[clin['Subgroup']!='HER2+']['Subgroup'].reset_index(drop=True)

In [4]:
folds_file = '../data/folds.pickle'
with open(folds_file, 'rb') as fid:
    outer_cross_val = pickle.load(fid)


In [5]:
X_train_list, y_train_list = [], []
X_val_list, y_val_list = [], []

for split in outer_cross_val:
    inner_cross_val, test_index = split
    for train_index, val_index in inner_cross_val:
        X_train, y_train = X.iloc[train_index,:], y[train_index]
        X_val, y_val = X.iloc[val_index,:], y[val_index]

        X_train_list.append(X_train)
        y_train_list.append(y_train)

        X_val_list.append(X_val)
        y_val_list.append(y_val)

In [6]:
accuracy_list = []
av_logloss_list = []
for (X_train, y_train, X_val, y_val) in zip(X_train_list, y_train_list, X_val_list, y_val_list):
    model = XGBClassifier(max_depth=2, learning_rate=1, objective='binary:logistic')
    model.fit(X_train, y_train=='HR+')

    y_pred = model.predict(X_val)
    predictions = [round(value) for value in y_pred]
    accuracy = accuracy_score(y_val=='HR+', predictions)
    av_logloss = log_loss(y_true=y_val=='HR+',labels=[True,False], y_pred=y_pred)

    accuracy_list.append(accuracy)
    av_logloss_list.append(av_logloss)

print(np.mean(accuracy_list),np.std(accuracy_list))
print(np.mean(av_logloss_list),np.std(av_logloss_list))


0.6973809523809524 0.18323420325537038
10.452219363763604 6.328776472873143
